<a href="https://colab.research.google.com/github/pstorniolo/Master2021/blob/main/2021_10_28_Loading_and_Saving_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and Saving Data in Spark

Collab Only code:

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!rm -f spark-3.2.0-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

#Install findspark using pip to make pyspark importable as regular library
!pip -q install findspark
import findspark
findspark.init()

Reading a text file textFile() in Python

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-3.2.0-bin-hadoop3.2/README.md")
lines.collect()

Loading all the .md files in one directory

In [ ]:
input = sc.textFile("spark-3.2.0-bin-hadoop3.2/*.md")

---
**Load file in Google Colab:**

In [ ]:
from google.colab import files
files.upload()

---

##Open**Data** Regione Siciliana
https://dati.regione.sicilia.it/dataset

####Load CSV from URL


In [ ]:
import requests

url = 'https://dati.regione.sicilia.it/download/dataset/arpa-qualita-aria-2019/filesystem/arpa-qualita-aria-2019-10_csv.zip'  
file_name = url.split('/')[-1]
r = requests.get(url, allow_redirects=True)
open(file_name,"wb").write(r.content)
print(file_name)
os.system("unzip "+file_name)
!rm -f *.zip; ls -la

###**Reading a CSV file into a DataFrame, filter some columns and save it**

In [ ]:
# Create DataFrame
df = spark.read.csv("arpa-qualita-aria-2019-10_csv.csv",sep=";",header=True,inferSchema=True)
df.printSchema()
df.show(5)

Filter data by several columns

In [ ]:
dataF=df.select("stazione_id","misura_dataora","misura_valore")
dataF.show()

Save only the filtered Data

In [ ]:
dataF.write.csv("dati_filtrati.csv",mode='overwrite',header=True)

Let's read this new file back into an RDD

In [ ]:
data = spark.read.csv("dati_filtrati.csv",header=True,inferSchema=True)
data.printSchema()
data.show()

In [ ]:
data.createOrReplaceTempView("table01")
spark.sql("show tables").show()

# **Hive Example**

Using Hive to create and read a table - Simple Example

In [ ]:
from pyspark.sql import HiveContext, Row

hiveCtx = HiveContext(sc)
ex1 = hiveCtx.read.csv("dati_filtrati.csv",header=True,inferSchema=True)

ex1.registerTempTable("Table02")
results = hiveCtx.sql("SELECT * FROM table02").show()

In [ ]:
spark.sql("show tables").show()

#Spark SQL

(https://spark.apache.org/docs/latest/sql-programming-guide.html)

**Reference**

(https://spark.apache.org/docs/latest/sql-ref.html)